### Train best configuration and model with all data available

In [1]:
### Entrenar con todo
import pandas as pd
import numpy as np

df_sets = pd.read_csv("../Datos/koi_sets.csv")
mask_total = df_sets["Set"] != "Unlabeled"
mask_unbl = df_sets["Set"] == "Unlabeled"

lc_total = np.load("../../Processed_Data/koi_light_curves_full.npy") #Raw light curve
#lc_total = np.load("../../Processed_Data/koi_light_curves_model_full.npy")
lc_total = lc_total[mask_total] 
lc_final = lc_total[mask_unbl]

df_label = pd.read_csv("../Datos/koi_metadata.csv")
df_label_total = df_label[mask_total] 
df_label_final = df_label[mask_unbl]
print("Read Done!")

In [ ]:
#labels??

In [2]:
lc_total.shape

(6257, 71427)

In [4]:
##Extraer caract con el modelo elegido
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import IncrementalPCA

#modelo
d=55
model =  IncrementalPCA(d)
std = StandardScaler(with_mean=True,with_std=False) #sin sacar varianza

##TODOS LOS DATOS##
Xfull_train = np.nan_to_num(Xfull_train)
#fourier
Xfull_fourier = np.abs(np.fft.fft(Xfull_train))
std.fit(Xfull_fourier)

Xfull_fourier = std.transform(Xfull_fourier)

#entrenar con full train (todo)
model.fit(Xfull_fourier)

##AJUSTAR LOS TRAIN Y CANDIDATES
X_train = np.nan_to_num(X_train)
X_candidates = np.nan_to_num(X_candidates)
#fourier
Xtrain_fourier = np.abs(np.fft.fft(X_train))
Xcandidates_fourier = np.abs(np.fft.fft(X_candidates))

Xpca_train = model.transform(Xtrain_fourier)
Xpca_candidate = model.transform(Xcandidates_fourier)

In [5]:
Xpca_train.shape

(6257, 55)

In [8]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
std.fit(asd)

Xstd_train = std.transform(asd)
Xstd_candidates = std.transform(asd)

In [20]:
#algoritmo elegido
from sklearn.svm import SVC
model = SVC(kernel= 'rbf',C=c,gamma='auto',class_weight = 'balanced')
model.fit(Xstd_train,ysample_train)

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [22]:
ypred_candidates = model.predict(Xstd_candidates) # o Xstd_candidates
ypred_candidates.shape

(1797,)

In [18]:
ypred_candidates

array([1, 1, 1, ..., 1, 1, 1])

In [23]:
print("Cantidad de Confirmados %d",np.sum(ypred_candidates))
print("Cantidad de Falsos positivos(noconfirmados) %d",(len(ypred_candidates)-np.sum(ypred_candidates)))

('Cantidad de Confirmados %d', 1708)
('Cantidad de Falsos positivos(noconfirmados) %d', 89)


In [24]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=15,max_depth=10,n_jobs=-1,class_weight = 'balanced') 
model.fit(asd,asd)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=18, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [25]:
ypred_candidates = model.predict(asd) 

In [26]:
print("Cantidad de Confirmados %d",np.sum(ypred_candidates))
print("Cantidad de Falsos positivos(noconfirmados) %d",(len(ypred_candidates)-np.sum(ypred_candidates)))

('Cantidad de Confirmados %d', 1259)
('Cantidad de Falsos positivos(noconfirmados) %d', 538)


In [ ]:
df_results = pd.Dataframe()
df_results["KOI Name"] = df_label_final["KOI Name"]
df_results["Our Disposition"] = model.predict(X)
df_results.head()

In [ ]:
#save!
df_results.to_csv("koi_disposition_2019.csv",index=False)